<a href="https://colab.research.google.com/github/JacopoMangiavacchi/TestCoreMLTools/blob/master/TF1/TestTensorFlow1x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==1.12.0
!pip install coremltools

     |████████████████████████████████| 83.1MB 74kB/s 
     |████████████████████████████████| 3.1MB 33.9MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 3.4MB 2.9MB/s 


In [3]:
import tensorflow as tf
print(tf.__version__)

1.12.0


In [0]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Input
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import preprocessing
import coremltools

# Prepare data

In [6]:
dataframe = pd.read_csv("./data/housing.csv", delim_whitespace=True, header=None)
dataframe.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [7]:
train_dataset = dataframe.sample(frac=0.8,random_state=0)
test_dataset = dataframe.drop(train_dataset.index)

train_dataset.shape, test_dataset.shape

((405, 14), (101, 14))

In [8]:
train_stats = train_dataset.describe()
train_stats.pop(13)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
0,405.0,3.670339,8.977954,0.00632,0.08265,0.26838,3.47428,88.9762
1,405.0,11.067901,22.688396,0.00000,0.00000,0.00000,12.50000,95.0000
2,405.0,11.097951,6.699157,0.74000,5.32000,9.69000,18.10000,27.7400
3,405.0,0.069136,0.253999,0.00000,0.00000,0.00000,0.00000,1.0000
4,405.0,0.553098,0.115564,0.38500,0.44900,0.53800,0.62400,0.8710
5,405.0,6.269768,0.690552,3.56100,5.87800,6.19300,6.61900,8.7800
6,405.0,68.241975,27.908527,2.90000,45.00000,76.50000,94.10000,100.0000
7,405.0,3.834098,2.162173,1.12960,2.07880,3.27210,5.21460,12.1265
8,405.0,9.474074,8.618564,1.00000,4.00000,5.00000,24.00000,24.0000
9,405.0,405.839506,164.057951,187.00000,284.00000,334.00000,666.00000,711.0000


In [9]:
train_labels = train_dataset.pop(13)
test_labels = test_dataset.pop(13)

train_dataset.shape, test_dataset.shape, train_labels.shape, test_labels.shape

((405, 13), (101, 13), (405,), (101,))

In [10]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

normed_train_data.shape, normed_test_data.shape, train_labels.shape, test_labels.shape

((405, 13), (101, 13), (405,), (101,))

In [0]:
train = normed_train_data.join(train_labels)
test = normed_test_data.join(test_labels)
X = normed_train_data.values
Y = train_labels.values

# Model

In [0]:
keras_model = Sequential([
    Dense(64, input_shape=[13], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1),
])

In [13]:
keras_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                896       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


# Save and Export no Train Model

In [0]:
keras_model.save('keras_no_train_model.h5')

In [15]:
mlmodel = coremltools.converters.keras.convert('keras_no_train_model.h5') # , use_float_arraytype = True)
mlmodel.save('coreml_model_double_no_train.mlmodel')

0 : dense_1_input, <keras.engine.input_layer.InputLayer object at 0x7f41bdb9add8>
1 : dense_1, <keras.layers.core.Dense object at 0x7f41bdb9ada0>
2 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f41bdb5dba8>
3 : dense_2, <keras.layers.core.Dense object at 0x7f41bdb9af98>
4 : dense_2__activation__, <keras.layers.core.Activation object at 0x7f41bdb5def0>
5 : dense_3, <keras.layers.core.Dense object at 0x7f41bdba6cc0>


Recommendation: This model has at least one multiarray input/output of type double.
For large sized arrays, multiarrays of type float32 are more efficient.
In future, float input/output multiarrays will be produced by default by the converter.
Please use, either the flag 'use_float_arraytype' during the call to convert or
the utility 'coremltools.utils.convert_double_to_float_multiarray_type(spec)', post-conversion.




/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Train Model

In [0]:
optimizer = keras.optimizers.RMSprop(0.001)  #Adam()

keras_model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

In [21]:
history = keras_model.fit(X, Y, epochs=500, verbose=1)

Epoch 1/500
405/405 [==============================] - 0s 308us/step - loss: 530.7045 - mean_absolute_error: 21.2605 - mean_squared_error: 530.7045
Epoch 2/500
405/405 [==============================] - 0s 46us/step - loss: 458.5354 - mean_absolute_error: 19.5586 - mean_squared_error: 458.5354
Epoch 3/500
405/405 [==============================] - 0s 47us/step - loss: 385.5517 - mean_absolute_error: 17.6334 - mean_squared_error: 385.5517
Epoch 4/500
405/405 [==============================] - 0s 46us/step - loss: 307.6333 - mean_absolute_error: 15.4750 - mean_squared_error: 307.6333
Epoch 5/500
405/405 [==============================] - 0s 40us/step - loss: 230.5435 - mean_absolute_error: 13.0359 - mean_squared_error: 230.5435
Epoch 6/500
405/405 [==============================] - 0s 45us/step - loss: 161.3204 - mean_absolute_error: 10.5219 - mean_squared_error: 161.3204
Epoch 7/500
405/405 [==============================] - 0s 46us/step - loss: 106.8344 - mean_absolute_error: 8.3092 - 

# Eval Model

In [25]:
loss, mae, mse = keras_model.evaluate(normed_test_data.values, test_labels.values, verbose=2)

print("Loss : {:5.2f}".format(loss))
print("Mae : {:5.2f}".format(mae))
print("Mse : {:5.2f}".format(mse))

Loss :  6.28
Mae :  1.86
Mse :  6.28


In [24]:
score = keras_model.predict(X)
print(score.shape)
print(score[0])
print(X[0], Y[0])

(405, 1)
[22.457115]
[-0.40132746 -0.48782211 -1.17297605 -0.27218959 -0.80559453  0.09156749
 -1.82890254  0.63847899 -0.63514919  0.14726805 -0.71781379  0.20738057
 -0.74734892] 22.6


# Test Prediction

In [52]:
print(normed_test_data.values[0], test_labels.values[0], keras_model.predict(np.array([normed_test_data.values[0]])))
print(normed_test_data.values[1], test_labels.values[1], keras_model.predict(np.array([normed_test_data.values[1]])))
print(normed_test_data.values[17], test_labels.values[17], keras_model.predict(np.array([normed_test_data.values[17]])))

[-0.38987719  0.06312032 -0.48184431 -0.27218959 -0.25178915 -0.38486309
  0.63271075  1.27556958 -0.51912059 -0.5780854  -1.51173314  0.33334089
  0.62918333] 18.9 [[19.53779]]
[-0.31519425 -0.48782211 -0.44154073 -0.27218959 -0.13064423 -0.97135058
  0.62554448  0.28698074 -0.63514919 -0.60246703  1.19693287 -0.57894038
  0.54597083] 13.9 [[15.569779]]
[-0.4044373  -0.48782211 -1.14759972 -0.27218959 -0.55465147  0.19583194
  0.20273463 -0.34317238 -0.86720641 -0.82799709 -0.29750355  0.40825989
 -0.62605612] 22.0 [[24.838232]]


# Save and Export Trained Model

In [27]:
keras_model.save('keras_train_model.h5')
mlmodel = coremltools.converters.keras.convert('keras_train_model.h5') # , use_float_arraytype = True)
mlmodel.save('coreml_model_double_train.mlmodel')

0 : dense_1_input, <keras.engine.input_layer.InputLayer object at 0x7f41bbf792e8>
1 : dense_1, <keras.layers.core.Dense object at 0x7f41bbf79c50>
2 : dense_1__activation__, <keras.layers.core.Activation object at 0x7f41bc81a748>
3 : dense_2, <keras.layers.core.Dense object at 0x7f41bbf79438>
4 : dense_2__activation__, <keras.layers.core.Activation object at 0x7f41bbe6ef98>
5 : dense_3, <keras.layers.core.Dense object at 0x7f41bbeeba58>


Recommendation: This model has at least one multiarray input/output of type double.
For large sized arrays, multiarrays of type float32 are more efficient.
In future, float input/output multiarrays will be produced by default by the converter.
Please use, either the flag 'use_float_arraytype' during the call to convert or
the utility 'coremltools.utils.convert_double_to_float_multiarray_type(spec)', post-conversion.




In [53]:
spec = mlmodel.get_spec()
spec

specificationVersion: 1
description {
  input {
    name: "input1"
    type {
      multiArrayType {
        shape: 13
        dataType: DOUBLE
      }
    }
  }
  output {
    name: "output1"
    type {
      multiArrayType {
        shape: 1
        dataType: DOUBLE
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.3"
    }
  }
}
neuralNetwork {
  layers {
    name: "dense_1"
    input: "input1"
    output: "dense_1_output"
    innerProduct {
      inputChannels: 13
      outputChannels: 64
      hasBias: true
      weights {
        floatValue: -0.43113693594932556
        floatValue: 0.12898887693881989
        floatValue: 0.2633260190486908
        floatValue: 0.018271714448928833
        floatValue: 0.24562415480613708
        floatValue: 0.1779996007680893
        floatValue: -0.3332698941230774
        floatValue: 0.027858933433890343
        floatValue: -0.1543409675359726
        floatValue: 0.04781841114163399
        floatValue